# Bitcoin Daily Trend Visualization (2024)

This notebook was inspired by a [Stack Overflow question](https://stackoverflow.com/questions/78801684/how-to-use-2-colors-on-same-line-lets-plot-in-kotlin).

In [1]:
%use dataframe
%useLatestDescriptors
%use lets-plot

In [2]:
LetsPlot.getInfo()

Lets-Plot Kotlin API v.4.12.0. Frontend: Notebook with dynamically loaded JS. Lets-Plot JS v.4.8.1.
Outputs: Web (HTML+JS), Kotlin Notebook (Swing), Static SVG (hidden)

In [3]:
import org.jetbrains.kotlinx.dataframe.io.readCsv

val year = 2024
val df = DataFrame.readCsv(fileOrUrl = "https://raw.githubusercontent.com/JetBrains/lets-plot-docs/refs/heads/master/data/Bitstamp_BTCUSD_d.csv")
    .filter { (it["date"] as LocalDateTime).year == year }
    .sortBy("date")
    .add("trend") { r ->
        val diff = r.diff(0f, expression = { (it["close"] as Number).toFloat() })
        if (diff > 0) 1 else 0
    }

df.head()

unix,date,symbol,open,high,low,close,Volume BTC,Volume USD,trend
1704067200,2024-01-01T00:00,BTC/USD,42258.000000,44192.000000,42188.000000,44187.000000,1633.498618,72179403.424287,0
1704153600,2024-01-02T00:00,BTC/USD,44187.000000,45922.000000,44167.000000,44970.000000,3948.659076,177571198.644572,1
1704240000,2024-01-03T00:00,BTC/USD,44972.000000,45510.000000,41454.000000,42865.000000,3980.828056,170638194.624727,0
1704326400,2024-01-04T00:00,BTC/USD,42863.000000,44795.000000,42656.000000,44186.000000,2918.119349,128940021.563309,1
1704412800,2024-01-05T00:00,BTC/USD,44195.000000,44366.000000,42500.000000,44184.000000,3155.466383,139421126.677960,0


In [4]:
(letsPlot(df.toMap())
        + geomLine(color = "pen", size = 1.2, tooltips = tooltipsNone) { x = "date"; y = "close" }
        + geomLine(size = .6,
            tooltips = layerTooltips().title("^x").format("^x", "%A, %b %e")
                .line("Close price|^y")
                .anchor("top_left")
                .minWidth(200)
        ) { x = "date"; y = "close"; color = "trend" }
        + scaleXDateTime(expand = listOf(0))
        + scaleYContinuous(format = "$.4~s")
        + scaleColorManual(
            name = "Trend",
            values = mapOf(0 to "red", 1 to "green"),
            labels = listOf("Downtrend", "Uptrend")
        )
        + ylab("Close Price")
        + ggtitle("Bitcoin Daily Close Price with Up/Down Trend Coloring ($year)")
        + ggsize(1000, 500)
        + themeLight()
        + flavorDarcula()
        + theme(plotTitle = elementText(size = 18, face = "bold"), axisTitleX = "blank")
            .legendPositionBottom()
        + ggtb()
    ).show()